# DALL·E 2 service with Azure Open AI

The DALL·E service offers an asynchronous API that lets you submit requests to generate images. It also includes Responsible AI text filters to prevent inappropriate images from being generated.

In [1]:
import gradio as gr
import openai
import os
import requests
import time

from dotenv import load_dotenv
from io import BytesIO
from PIL import Image

In [2]:
def dalle2(prompt, size=1024):
    """
    Generate an image from a prompt with Dall e 2
    """
    # prompt
    extra_prompt = "full view, detailled, 8K"
    image_prompt = prompt + extra_prompt

    # Get the endpoint and key for Azure Open AI
    load_dotenv("azure.env")
    openai.api_base = os.getenv("azure_openai_endpoint")
    openai.api_key = os.getenv("azure_openai_key")

    api_version = "2023-06-01-preview"
    url = (
        f"{openai.api_base}/openai/images/generations:submit?api-version={api_version}"
    )
    headers = {
        "api-key": openai.api_key,
        "Content-Type": "application/json",
    }

    if size == 512:
        imagesize = "512x512"
    elif size == 1024:
        imagesize = "1024x1024"
    elif size == 256:
        imagesize = "256x256"
    else:
        imagesize = "1024x1024"

    body = {"prompt": image_prompt, "size": imagesize, "n": 1}

    submission = requests.post(url, headers=headers, json=body)
    operation_location = submission.headers["operation-location"]
    status = ""

    while status != "succeeded":
        time.sleep(1)
        response = requests.get(operation_location, headers=headers)
        status = response.json()["status"]

    # Get the first generated image
    image_url = response.json()["result"]["data"][0]["url"]
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))

    return img

In [3]:
logo = "https://github.com/retkowsky/images/blob/master/aoai.jpg?raw=true"
image = "<center> <img src= {} width=240px></center>".format(logo)
title = "Text to image with Azure OpenAI Dalle2"

samples = [
    "Two people walking in the street in the style of Basquiat",
    "A lady hiding her eyes, polaroid style",
    "A boy playing football, pixel style",
    "Portrait of a lady with long hair, in the style of Andy Warhol",
    "Portrait, ages 20-40",
    "A rainbow over a snowy mountain",
    "Distant view of some scratches on the door of a car",
    "Pixar style 3D render of a baby hippo, 4k, trending in artstation",
    "A satellite view of a lake and forests",
    "A sailbot in the sea, watercolor",
]

inputs = [gr.Textbox(label="Your prompt to generate the artificial image")]
outputs = [gr.Image(label="The generated image", type="pil")]


# Webapp
webapp = gr.Interface(
    fn=dalle2,
    inputs=inputs,
    outputs=outputs,
    description=image,
    title=title,
    examples=samples,
    theme="gradio/monochrome",  #  https://huggingface.co/spaces/gradio/theme-gallery
)

# Running the webapp
webapp.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://dfef2ac1bdbd0895e5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
